In [1]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [2]:
import tensorflow as tf

In [3]:
# GLobal definations

path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h, block_w = (32, 32)
Sub_Model = 5

In [4]:
# Function Definition --> Loading preprocessed data from the file & Normalize it.

def Load_Data():
    pickle_in = open(Case_Path + '/' + 'X_train.pickle', 'rb')
    X = pickle.load(pickle_in)  # Loading Features

    pickle_in = open(Case_Path + '/' + 'y_train.pickle', 'rb')
    y = pickle.load(pickle_in)  # Loading Labels

    print('Shape of X:\t', X.shape), print('Shape of y:\t', y.shape)

    return X, y

In [5]:
# Function Definition --> For Development, Compilation, Training & Storing of the Model

def Model(img_h, img_w, X_train, y_train, X_test, y_test, count):
    # import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Activation
    from tensorflow.keras.utils import to_categorical

    model = Sequential()

    model.add (Conv2D (32, (3, 3), padding = 'same', input_shape = (img_h, img_w, 1)))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (MaxPooling2D (pool_size = (3, 3)))  # Pool_size is the size of filter.
    model.add (Dropout(0.1))

    model.add (Conv2D (48, (3, 3), padding = 'same'))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (MaxPooling2D (pool_size = (3, 3)))
    model.add (Dropout(0.1))

    model.add (Conv2D (48, (3, 3), padding = 'same'))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (MaxPooling2D (pool_size = (3, 3)))
    model.add (Dropout(0.1))

    model.add (Flatten())  # Convert 3D feature map to 1D feature vector.

    model.add (Dense(1096))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (Dropout(0.1))
    model.add (Dense(2, activation = 'softmax'))

    # Compiling the model
    model.compile (loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.optimizer.learning_rate = 0.0001

    # Training the model & storing its detail in the variable h
    history = model.fit (X_train, to_categorical(y_train), batch_size = 128, epochs = 50, verbose = 2,
               validation_data=(X_test, to_categorical(y_test)), shuffle = True)

    # Evaluating performance of the Model
    test_loss, test_acc = model.evaluate (X_test, to_categorical(y_test))
    results.append(test_acc)
    print('Test Loss is: ', test_loss, '\nTest Accuracy is: ', test_acc)

    # Saving the model in hierarchical Data Formate (HDF)
    model.save (path + 'Outputs/Model_' + case + '_' + str(count+1) + '.h5')

    return history, model

In [6]:
# Function Definition --> For Saving Validation Accuracy in the file

def Saving_Test_Accuracy():
    import numpy as np
    
    file = open(Result_File, "a")
    file.write(case)
    file.write(':\n======')
    file.write('\nAll Test Accuracy:\t')
    
    for ans in results:
        file.write(str(ans) + ', ')
    
    file.write('\nMean Test Accuracy:\t' + str(np.mean(results)))
    file.write('\nMin Test Accuracy:\t' + str(np.min(results)))
    file.write('\nMax Test Accuracy:\t' + str(np.max(results)))
    file.write('\n\n\n')
    file.close()

In [9]:
# All Function Calls
import os
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split
from random import randint

PATH = path + 'Data/'
Work_Dir = PATH + 'Working_Data/'
Result_File = path + 'Outputs/Results.txt'

# Calling Functions:
osdir=os.listdir(Work_Dir)
if '.DS_Store' in osdir:
    osdir.remove('.DS_Store')

for case in tqdm(osdir):
    Case_Path = os.path.join(Work_Dir, case)  # Joining path for "Cases'.
    print(case)

    # Function Call --> Loading preprocessed data from the file & Normalize it.
    X, y = Load_Data()

    results = []
    for i in range(Sub_Model):
        print('\n' + case + '_' + str(i+1))
        
        # Spliting data into two parts, "Training = 70%" and "Testing = 30%".
        num = randint(0,1000)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = num)

        # Function Call --> For Development, Compilation, Training & Storing of the Model
        history, model = Model(block_h, block_w, X_train, y_train, X_test, y_test, i)
    print('\n')

    # Function Call --> For Saving Validation Accuracy in the file
    Saving_Test_Accuracy()
print('Done...!!')

  0%|          | 0/5 [00:00<?, ?it/s]

Case_5
Shape of X:	 (32148, 32, 32, 1)
Shape of y:	 (32148,)

Case_5_1
Epoch 1/50


2022-09-05 15:53:05.032669: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-05 15:53:08.125642: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


176/176 - 4s - loss: 1.0195 - accuracy: 0.5179 - val_loss: 0.6954 - val_accuracy: 0.5247 - 4s/epoch - 22ms/step
Epoch 2/50
176/176 - 3s - loss: 0.7098 - accuracy: 0.5419 - val_loss: 0.6725 - val_accuracy: 0.5859 - 3s/epoch - 19ms/step
Epoch 3/50
176/176 - 3s - loss: 0.6869 - accuracy: 0.5676 - val_loss: 0.6695 - val_accuracy: 0.6321 - 3s/epoch - 18ms/step
Epoch 4/50
176/176 - 3s - loss: 0.6806 - accuracy: 0.5783 - val_loss: 0.6580 - val_accuracy: 0.6331 - 3s/epoch - 19ms/step
Epoch 5/50
176/176 - 3s - loss: 0.6700 - accuracy: 0.5959 - val_loss: 0.6555 - val_accuracy: 0.5767 - 3s/epoch - 19ms/step
Epoch 6/50
176/176 - 3s - loss: 0.6642 - accuracy: 0.6058 - val_loss: 0.6424 - val_accuracy: 0.6507 - 3s/epoch - 19ms/step
Epoch 7/50
176/176 - 3s - loss: 0.6532 - accuracy: 0.6197 - val_loss: 0.6304 - val_accuracy: 0.6442 - 3s/epoch - 18ms/step
Epoch 8/50
176/176 - 3s - loss: 0.6443 - accuracy: 0.6243 - val_loss: 0.6314 - val_accuracy: 0.6057 - 3s/epoch - 18ms/step
Epoch 9/50
176/176 - 3s - l

2022-09-05 15:55:50.041547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-05 15:55:53.028515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

176/176 - 4s - loss: 0.9903 - accuracy: 0.5208 - val_loss: 0.6857 - val_accuracy: 0.5808 - 4s/epoch - 25ms/step
Epoch 2/50
176/176 - 4s - loss: 0.7046 - accuracy: 0.5554 - val_loss: 0.6688 - val_accuracy: 0.6401 - 4s/epoch - 20ms/step
Epoch 3/50
176/176 - 3s - loss: 0.6908 - accuracy: 0.5636 - val_loss: 0.6756 - val_accuracy: 0.5742 - 3s/epoch - 20ms/step
Epoch 4/50
176/176 - 4s - loss: 0.6783 - accuracy: 0.5798 - val_loss: 0.6492 - val_accuracy: 0.6202 - 4s/epoch - 20ms/step
Epoch 5/50
176/176 - 3s - loss: 0.6643 - accuracy: 0.6032 - val_loss: 0.6400 - val_accuracy: 0.6371 - 3s/epoch - 20ms/step
Epoch 6/50
176/176 - 4s - loss: 0.6553 - accuracy: 0.6152 - val_loss: 0.6312 - val_accuracy: 0.6436 - 4s/epoch - 20ms/step
Epoch 7/50
176/176 - 3s - loss: 0.6529 - accuracy: 0.6183 - val_loss: 0.6415 - val_accuracy: 0.5889 - 3s/epoch - 20ms/step
Epoch 8/50
176/176 - 4s - loss: 0.6320 - accuracy: 0.6407 - val_loss: 0.6113 - val_accuracy: 0.6440 - 4s/epoch - 20ms/step
Epoch 9/50
176/176 - 4s - l

2022-09-05 17:08:23.286614: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-05 17:08:26.861603: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


176/176 - 5s - loss: 1.0034 - accuracy: 0.5233 - val_loss: 0.7172 - val_accuracy: 0.4687 - 5s/epoch - 26ms/step
Epoch 2/50
176/176 - 4s - loss: 0.7052 - accuracy: 0.5504 - val_loss: 0.6674 - val_accuracy: 0.6382 - 4s/epoch - 20ms/step
Epoch 3/50
176/176 - 4s - loss: 0.6952 - accuracy: 0.5598 - val_loss: 0.6634 - val_accuracy: 0.6355 - 4s/epoch - 20ms/step
Epoch 4/50
176/176 - 4s - loss: 0.6769 - accuracy: 0.5871 - val_loss: 0.6718 - val_accuracy: 0.5411 - 4s/epoch - 20ms/step
Epoch 5/50
176/176 - 4s - loss: 0.6731 - accuracy: 0.5904 - val_loss: 0.6437 - val_accuracy: 0.6396 - 4s/epoch - 20ms/step
Epoch 6/50
176/176 - 4s - loss: 0.6588 - accuracy: 0.6111 - val_loss: 0.6421 - val_accuracy: 0.6074 - 4s/epoch - 20ms/step
Epoch 7/50
176/176 - 4s - loss: 0.6492 - accuracy: 0.6206 - val_loss: 0.6818 - val_accuracy: 0.5565 - 4s/epoch - 21ms/step
Epoch 8/50
176/176 - 4s - loss: 0.6435 - accuracy: 0.6273 - val_loss: 0.6745 - val_accuracy: 0.5678 - 4s/epoch - 20ms/step
Epoch 9/50
176/176 - 4s - l

100%|██████████| 5/5 [1:18:20<00:00, 940.09s/it]



Done...!!
